In [1]:
import modules.labelHelper as lh 
import modules.featureHelper as fh
from keras.callbacks import (ModelCheckpoint, LearningRateScheduler,
                             EarlyStopping, ReduceLROnPlateau,CSVLogger)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import warnings
import keras
from keras.callbacks import EarlyStopping
from keras.regularizers import l2 
from keras.models import Sequential
from keras.layers import Reshape
from keras.layers import Dense, Dropout, Activation, Flatten,Conv1D, GlobalAveragePooling1D, MaxPooling1D,AveragePooling1D
from keras.layers import Conv2D, MaxPooling2D,Conv1D,GlobalMaxPooling1D,MaxPooling1D,average, concatenate,RepeatVector,Lambda,add,subtract,Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Input, Model
from sklearn import metrics as mt
from skimage.io import imshow
from sklearn import preprocessing
from matplotlib.patches import Rectangle
from keras import backend as K
from keras.layers.normalization import BatchNormalization
warnings.filterwarnings('ignore')
from modules.modelTools import *

Import an awesome Label helper
import an awesome feature helper


C:\Users\razer\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\razer\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\razer\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])

In [2]:
ll=lh.Label('..//2019_fall_labels.csv')
print("There are %d videos in total\n"%ll.video_count())
ll.get_trainee_info()

train_video_number,test_video_number=ll.train_test_split_on_people(0.2)
print(train_video_number)
print(test_video_number)

## set up feature parameters ##
folder = "C:\\2019_fall_video_features"
video_clips_length=30
time_lag=2
move_threshold=150
stride=video_clips_length

# train test split by people
train_video,train_video_label=fh.make_train_test_data_from_video_numbers(folder,train_video_number,video_clips_length,time_lag,move_threshold,stride,ll)
test_video,test_video_label=fh.make_train_test_data_from_video_numbers(folder,test_video_number,video_clips_length,time_lag,move_threshold,stride,ll)
print(train_video.shape)
print(train_video_label.shape)
print(test_video.shape)
print(test_video_label.shape)

There are 83 videos in total

There are 19 trainees in total

 Number of Videos from each person
Akhtar          -- [176 253 761] ----: 3
Cadeddu         -- [164 171] ----: 2
Crivelli        -- [225 276 333 425] ----: 4
Gahan           -- [257 345 378 384 486] ----: 5
Johnson         -- [441 615] ----: 2
Keith           -- [240 277 717] ----: 3
Kenigsberg      -- [ 78 440 455 471 881] ----: 5
Krabbe          -- [152 344 527 861] ----: 4
Marthur         -- [130 194 222 368 921] ----: 5
Mollengarden    -- [ 59 143 267 460] ----: 4
Moony           -- [294 301 361 498 539] ----: 5
Passoni         -- [207 237 895 942] ----: 4
Rozanski        -- [113 302 716] ----: 3
Satyanarayan    -- [ 16  74 236 358 436 457 503 537 557 578 599 632 689] ----: 13
Singla          -- [ 49 536 538] ----: 3
Sorokin         -- [ 91 226 507 530] ----: 4
Timburlake      -- [258 296 559 742] ----: 4
Tse             -- [ 11 283 414 427 562 928] ----: 6
Varun           -- [192 401 417 820] ----: 4
train on videos fro

In [3]:
# model.summary()

In [6]:
X_train=train_video/640-0.5
X_test=test_video/640-0.5
y_train=train_video_label/5
y_test=test_video_label/5
#     y_train=y_train
#     y_test=y_test
# print(X_train.shape,y_train.shape)
# print(X_test.shape,y_test.shape)

# DP = []  
# BD = []
# E = []
# FS = []
# A = []
# RC = []
# for i in range(len(y_test)):
#     DP.append(y_test[i][0])
#     BD.append(y_test[i][1])
#     E.append(y_test[i][2])
#     FS.append(y_test[i][3])
#     A.append(y_test[i][4])
#     RC.append(y_test[i][5])
# y_test_labels = [np.array(DP),np.array(BD),np.array(E),np.array(FS), np.array(A),np.array(RC)]
# DP = []  
# BD = []
# E = []
# FS = []
# A = []
# RC = []
# for i in range(len(y_train)):
#     DP.append(y_train[i][0])
#     BD.append(y_train[i][1])
#     E.append(y_train[i][2])
#     FS.append(y_train[i][3])
#     A.append(y_train[i][4])
#     RC.append(y_train[i][5])
# y_trin_labels = [np.array(DP),np.array(BD),np.array(E),np.array(FS), np.array(A),np.array(RC)]
sample_of_trainningdata=X_train.shape[0]
sample_of_testdata=X_test.shape[0]

batch_size=64

train_gen=generator(X_train, y_train, batch_size,0.1)
val_gen=generator(X_test, y_test, batch_size,0)

#prepare for inout shape
clip_lenth=X_train.shape[1]
dimension=X_train.shape[2]
l2_lambda=0.01
model=make_model(0.01,clip_lenth,dimension)
callbacks_list = get_callback_list_by_model('MTmodel')
# model_h=model.fit_generator(train_gen,
#                     steps_per_epoch=sample_of_trainningdata//batch_size,
#                     epochs=30,
#                     validation_data=val_gen,
#                    validation_steps=sample_of_testdata//batch_size,
#                     verbose=1,
#                     callbacks=callbacks_list
#  )
# model.fit(x=X_train, # create a list of inputs for embeddings
#         y=y_trin_labels, epochs=30, 
#         batch_size=16, verbose=1,
#         validation_data = (X_test,y_test_labels),
#         callbacks=callbacks_list         
#            )

Train on 2320 samples, validate on 382 samples
Epoch 1/30
2320/2320 [==============================] - 9s 4ms/step - loss: 2.2203 - DP_loss: 0.4238 - BD_loss: 0.3415 - E_loss: 0.3661 - FS_loss: 0.3173 - A_loss: 0.3609 - RC_loss: 0.3649 - val_loss: 2.2266 - val_DP_loss: 0.5064 - val_BD_loss: 0.3843 - val_E_loss: 0.4165 - val_FS_loss: 0.2308 - val_A_loss: 0.3404 - val_RC_loss: 0.3023

Epoch 00001: val_loss improved from inf to 2.22661, saving model to model_zoo/MTmodel.h5
Epoch 2/30
2320/2320 [==============================] - 3s 1ms/step - loss: 1.5152 - DP_loss: 0.2474 - BD_loss: 0.2574 - E_loss: 0.3097 - FS_loss: 0.2218 - A_loss: 0.2149 - RC_loss: 0.2179 - val_loss: 1.2602 - val_DP_loss: 0.1940 - val_BD_loss: 0.1737 - val_E_loss: 0.3758 - val_FS_loss: 0.1289 - val_A_loss: 0.1512 - val_RC_loss: 0.1906

Epoch 00002: val_loss improved from 2.22661 to 1.26017, saving model to model_zoo/MTmodel.h5
Epoch 3/30
2320/2320 [==============================] - 3s 1ms/step - loss: 1.0948 - DP_loss:

In [7]:
model.load_weights('model_zoo/MTmodel.h5')

In [8]:
def model_predit_by_videoNumber(model,folder,vNum,video_clips_length,time_lag,move_threshold,stride,label):
    vNum=[vNum]
    video,video_label = fh.make_train_test_data_from_video_numbers(folder,vNum,video_clips_length,time_lag,move_threshold,stride,label)
    result = model.predict(video)
    return result,video_label
    

In [9]:
re,gt =model_predit_by_videoNumber(model,folder,16,video_clips_length,time_lag,move_threshold,stride,ll)

load feature from files, there are 1 videos
after deop:  1


In [ ]:
re[0].shape

In [10]:
re

[array([[    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [ 1574.6099],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [15929.724 ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [15400.154 ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [    0.    ],
        [ 